In [13]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [6]:
dir_int = os.path.abspath('../data/interim')
final_csv = os.path.join(dir_int, 'block_group_final.csv')

gdb_int = os.path.join(dir_int, 'interim.gdb')
final_fc = os.path.join(gdb_int, 'block_group_final')

In [9]:
final_df = pd.read_csv(final_csv)
final_df.drop('SHAPE', inplace=True, axis=1)
final_df.sample(5)

,origin_id,gender_pop0_cy,gender_pop5_cy,gender_pop10_cy,gender_pop15_cy,gender_pop20_cy,gender_pop25_cy,gender_pop30_cy,gender_pop35_cy,gender_pop40_cy,...,trip_time_minutes_OTHER_03,trip_time_minutes_STARBUCKS_01,trip_time_minutes_STARBUCKS_02,trip_time_minutes_STARBUCKS_03,trip_time_minutes_STUMPTOWN_COFFEE_ROASTERS_01,trip_time_minutes_STUMPTOWN_COFFEE_ROASTERS_02,trip_time_minutes_STUMPTOWN_COFFEE_ROASTERS_03,trip_time_minutes_UNCLE_GARYS_COFFEE_EMPORIUM_01,trip_time_minutes_UNCLE_GARYS_COFFEE_EMPORIUM_02,trip_time_minutes_UNCLE_GARYS_COFFEE_EMPORIUM_03
4696,530330096001,39.0,49.0,48.0,38.0,28.0,23.0,35.0,58.0,53.0,...,NaN,169.733333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6647,530050114012,72.0,74.0,83.0,82.0,62.0,80.0,85.0,73.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6626,530050110021,106.0,95.0,72.0,62.0,105.0,136.0,127.0,91.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2881,530419703003,71.0,69.0,65.0,66.0,75.0,80.0,70.0,57.0,60.0,...,86.250000,89.925000,120.666667,90.00,NaN,NaN,NaN,82.750000,108.916667,NaN
1947,410050218011,94.0,106.0,117.0,110.0,89.0,80.0,101.0,128.0,126.0,...,12.416667,6.216667,31.800000,13.25,NaN,NaN,NaN,18.666667,9.083333,NaN


In [14]:
X = final_df
label_col = 'market_penetration_UNCLE_GARYS_COFFEE_EMPORIUM_01'

# while possibly useful for post analysis, the destination id columns are not needed for modeling
start_df = X[[col for col in X.columns if not col.startswith('dest_id') and col is not 'SHAPE']]

# only keep rows with values for our label
start_df = start_df[start_df[label_col].notnull()].copy()

# extract out the labels and factors into X and y, respectively
y = start_df[label_col]
X = start_df[[col for col in start_df if col is not label_col]]

prep_pipe = Pipeline([
    ('impute_mean', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('scaler', MinMaxScaler())
])

X = prep_pipe.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestRegressor

rnd_frst = RandomForestRegressor(n_estimators=100)

In [17]:
rnd_frst.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [19]:
y_hat = rnd_frst.predict(X_test)

In [20]:
from sklearn.metrics import mean_squared_error

rnd_frst_mse = mean_squared_error(y_test, y_hat)
rnd_frst_rmse = np.sqrt(rnd_frst_mse)
rnd_frst_rmse

0.023591126958684882